In [ ]:
import sys
from itertools import combinations, product 
from pathlib import Path

import pandas as pd
from scipy.stats import spearmanr, kendalltau, pearsonr
from tqdm.notebook import tqdm

from constants import exclude_models, exclude_models_w_mae
from helper import load_model_configs_and_allowed_models, load_similarity_matrices

sys.path.append('..')
from scripts.helper import parse_datasets
from constants import sim_metric_name_mapping, anchors

In [ ]:
base_path_similarity_matrices = Path('/home/space/diverse_priors/model_similarities')
sim_metrics = [
    'cka_kernel_rbf_unbiased_sigma_0.4',
    'cka_kernel_linear_unbiased',
    'rsa_method_correlation_corr_method_spearman',
]

corr_methods = ['spearmanr', 'pearsonr', 'kendalltau']

combs = list(product(corr_methods, [exclude_models, exclude_models_w_mae]))
print(f"{len(combs)=}")
corr_method, curr_model_excl = combs[5]
print(f"{corr_method=}, {curr_model_excl=}")

suffix = '_wo_swav_pirl_timm_clip' if len(curr_model_excl)==3 else '_wo_mae_swav_pirl_timm_clip'


ds_list = parse_datasets('../scripts/webdatasets_w_insub10k.txt')
ds_list = list(map(lambda x: x.replace('/', '_'), ds_list))

storing_path = Path('/home/space/diverse_priors/results/aggregated/r_coeff_dist')
storing_path.mkdir(parents=True, exist_ok=True)

In [ ]:
model_configs, allowed_models = load_model_configs_and_allowed_models(
    path='../scripts/models_config_wo_barlowtwins_n_alignment.json',
    exclude_models=curr_model_excl,
    exclude_alignment=True,
)

In [ ]:
if 'mae' in suffix:
    anchors.remove('mae-vit-large-p16')

In [ ]:
model_configs['objective'].value_counts()

In [ ]:
sim_mats = load_similarity_matrices(
    path=base_path_similarity_matrices,
    ds_list=ds_list,
    sim_metrics=sim_metrics,
    allowed_models=allowed_models,
)

In [ ]:
anchor_col = 'Anchor Model'
other_col = 'Other Model'
other_ds_col = 'Dataset'
sim_metric_col = 'Similarity metric'
sim_ds_col = 'Similarity value DS'
# sim_imgnet_col = 'Similarity value IN'
info_orig_cols = ['objective', 'architecture_class', 'dataset_class', 'size_class']
info_cols = ['Objective', 'Architecture', 'Dataset size', 'Model size']
id_cols = [anchor_col, other_col, sim_metric_col] + info_cols
comp_cat_col = 'Comparison category'
comp_cat_orig_col = 'Comparison category (orig. name)'
comp_val_col = 'Comparison values'
r_col = 'r coeff'


def get_other_model_info(mid):
    model_config = model_configs.loc[mid]
    return model_config['objective'], model_config['architecture_class'], model_config['dataset_class'], model_config[
        'size_class']


def get_melted_sim_values_metric_anchor(anch, met, met_ds_mats):
    sim_vals_ds = []
    # for each dataset, get the similarity values of the anchor model to all other models
    for ds, curr_sim_mat in met_ds_mats.items():
        cols = curr_sim_mat.columns.tolist()
        cols.remove(anch)
        cols = sorted(list(set(cols).intersection(allowed_models)))
        row_sim_mat = curr_sim_mat.loc[anch, cols]
        row_sim_mat.name = ds
        sim_vals_ds.append(row_sim_mat)

    # concatenate the similarity values of the anchor model to all other models for all datasets
    anchor_sim_vals = pd.concat(sim_vals_ds, axis=1)
    anchor_sim_vals = anchor_sim_vals.reset_index(names=[other_col])
    # get the model information for all other models and add it to the dataframe
    anchor_sim_vals = pd.concat([anchor_sim_vals,
                                 pd.DataFrame(anchor_sim_vals[other_col].apply(get_other_model_info).tolist(),
                                              columns=info_cols)], axis=1)
    anchor_sim_vals[sim_metric_col] = sim_metric_name_mapping[met]
    anchor_sim_vals[anchor_col] = anch
    # melt the dataframe to have one row per dataset and anchor & other model pair 
    anchor_sim_vals = pd.melt(anchor_sim_vals,
                              id_vars=id_cols,
                              var_name=other_ds_col,
                              value_name=sim_ds_col,
                              )
    return anchor_sim_vals

In [ ]:
# get the similarity values for all anchor models and similarity metrics
dfs = []
for anchor in anchors:
    for sim_metric, ds_sim_mat in sim_mats.items():
        anchor_sim_vals = get_melted_sim_values_metric_anchor(anchor, sim_metric, ds_sim_mat)
        dfs.append(anchor_sim_vals)

all_sims = pd.concat(dfs, axis=0).reset_index(drop=True)

In [ ]:
all_sims.head()

In [ ]:
# get all unique datasets and create all possible pairs
all_datasets = all_sims[other_ds_col].unique()
ds_pairs = list(combinations(all_datasets, 2))
print('Nr. dataset pairs: ', len(ds_pairs))

In [ ]:
def correlation(x,y):
    if corr_method == 'spearmanr':
        corr, _ = spearmanr(x, y)
    elif corr_method == 'kendalltau':
        corr, _ = kendalltau(x, y)
    elif corr_method == 'pearsonr':
        corr, _ = pearsonr(x, y)
    return corr


def compute_corr(data):
    res = []
    for ds1, ds2 in ds_pairs:
        x = data[data[other_ds_col] == ds1][sim_ds_col]
        y = data[data[other_ds_col] == ds2][sim_ds_col]
        corr = correlation(x, y)
        res.append((ds1, ds2, corr))
    return res


def post_process_group_op(df):
    df = df.explode(r_col).reset_index(drop=True)
    df_split = df[r_col].apply(pd.Series)
    df_split.columns = ['DS 1', 'DS 2', r_col]
    df.drop(columns=[r_col], inplace=True)
    df = pd.concat([df, df_split], axis=1)
    return df


r_dfs = []
for strata in tqdm(info_cols):
    grouping_cols = [sim_metric_col, anchor_col, strata]

    strata_rs = all_sims.groupby(grouping_cols, dropna=False).apply(compute_corr, include_groups=False).reset_index()
    strata_rs.columns = grouping_cols + [r_col]
    strata_rs = post_process_group_op(strata_rs)

    all_rs = all_sims.groupby(grouping_cols[:-1], dropna=False).apply(compute_corr, include_groups=False).reset_index()
    all_rs.columns = grouping_cols[:-1] + [r_col]
    all_rs = post_process_group_op(all_rs)
    all_rs[strata] = 'All'

    rs = pd.concat([all_rs, strata_rs], axis=0).reset_index(drop=True)
    rs = rs.sort_values([sim_metric_col, anchor_col]).reset_index(drop=True)
    r_dfs.append(rs)

In [ ]:
for i in range(len(r_dfs)):
    r_dfs[i][comp_cat_col] = info_cols[i]
    r_dfs[i][comp_cat_orig_col] = info_orig_cols[i]
    r_dfs[i].rename(columns={info_cols[i]: comp_val_col}, inplace=True)

In [ ]:
r_df = pd.concat(r_dfs, axis=0)
r_df.head()

In [ ]:
r_df.to_csv(storing_path / f'agg_{corr_method}_all_ds{suffix}.csv', index=False)
print(f"Stored aggregated data at: {storing_path / f'agg_{corr_method}_all_ds{suffix}.csv'}")